# Classes

In [1]:
class Player:
    
    '''
    Base class used to instantiate the dealer.
    '''
    
    def __init__(self, name):
        self.name = name
        
    def __str__(self):
        return f'{self.name}'

In [2]:
class HumanPlayer(Player):
    
    '''
    Class derived from base class Player. Additional attribute for bankroll balance for the bets placed by the human player.
    '''
    
    def __init__(self, name, bankroll):
        Player.__init__(self, name)
        self.bankroll = bankroll
        print(f'Your bankroll balance is {self.bankroll}.')
        
    def __str__(self):
        return f'Player: {self.name}, Bankroll Balance: {self.bankroll}.'

In [3]:
class Card:
    
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
        
    def __str__(self):
        return self.rank + ' of ' + self.suit

In [4]:
class Deck:
    
    def __init__(self):
        self.deck = []
        for suit in suits_tpl:
            for rank in ranks_tpl:
                self.deck.append(Card(rank, suit))
    
    def __str__(self):
        deck_str = ''
        for card in self.deck:
            deck_str += '\n' + card.__str__()
        return 'The deck consists of: '+ deck_str
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def deal(self):
        return self.deck.pop()

In [5]:
class Hand:
    
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0
                
    def __str__(self):
        hand_str = ''
        for card in self.cards:
            hand_str += '\n' + card.__str__()
        hand_str += f'\nThe total hand value is {self.value}'
        return hand_str
    
    def add_card(self, card):
        self.cards.append(card)
        self.value += values_dct[card.rank]
        
        if card.rank == 'Ace':
            self.aces += 1
    
    def initial_deal(self):
        for i in range(1,3):
            self.add_card(deck.deal())
            
    def adjust_for_aces(self):
        if self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1
            
    def check_value(self):
        while self.value > 21 and self.aces:
            self.adjust_for_aces()
            print('Adjusted for an ace!')
            
        if self.value == 21:
            print('Black Jack!')
            return False
        elif self.value > 21:
            print('Bust!')
            return False
        else:
            return True

# Game functions

In [6]:
def get_bet_value():
    while True:
        try:
            bet_value = int(input(f'Your current balance is: {player1.bankroll}. How much do you want to bet?: '))
        except:
            print('Invalid value entered.')
        else:
            if bet_value > player1.bankroll:
                print('You don\'t have sufficient credit. Enter a lower value.')
            else:
                break

    return bet_value

In [7]:
def compare_player_dealer_hand(player_hand, dealer_hand):
    if player_hand.value < dealer_hand.value:
        print('The dealer wins!')
        return False
    else:
        return True

# Game logic

In [8]:
import random

suits_tpl = ('Hearts', 'Diamonds', 'Clubs', 'Spades')
ranks_tpl = ('2', '3', '4', '5', '6', '7', '8', '9', '10',
             'Jack', 'Queen', 'King', 'Ace')
values_dct = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
              '7': 7, '8': 8, '9': 9, '10': 10, 'Jack': 10,
              'Queen': 10, 'King': 10, 'Ace': 11}

game_on = True

player_name = input('What is your name: ')
player1 = HumanPlayer(player_name, 100)
dealer = Player('Dealer')

while game_on: # game
    
    win = False
    
    player1_hand = Hand()
    dealer_hand = Hand()
    
    deck = Deck()
    deck.shuffle()

    # player places bet
    bet_value = get_bet_value()
    
    # player gets two cards, cards are displayed
    player1_hand.initial_deal()
    
    while game_on: # player's turn

        #check player hand's value
        game_on = player1_hand.check_value()
        
        if not game_on:
            break

        print(f'\nYou hold {player1_hand.__str__()}.')
        choice = input('Do you want to hit or stay? ')

        if choice == 'stay':
            break
        elif choice == 'hit':
            dealt_card = deck.deal()
            player1_hand.add_card(dealt_card)
            print(f'You were dealt {dealt_card.__str__()}.')
        else:
            print('Invalid input. Please enter hit or stay.')

    if not win: # dealer's turn
        
        print("\nNow it's the dealer's turn.")

        # dealer gets two cards, cards are displayed
        dealer_hand.initial_deal()

        game_on = True

        while game_on:
            
            print(f'\nThe dealer holds {dealer_hand.__str__()}.')
            
            # dealer hits until exceeds player score <21, hits 21, or goes bust
            game_on = dealer_hand.check_value()
            
            if not game_on:
                win = True # player wins
                break

            game_on = compare_player_dealer_hand(player1_hand, dealer_hand)

            if not game_on:
                win = False # dealer wins
                break

            dealt_card = deck.deal()
            dealer_hand.add_card(dealt_card)
            print(f'The dealer was dealt {dealt_card.__str__()}.')

    if skip or win: # player wins
        print('You won. You receive double your bet.')
        player1.bankroll += bet_value
    elif not win: # dealer wins
        print('You lost your bet.')
        player1.bankroll -= bet_value

    # player can decide to play again or end the game
    if player1.bankroll:
        while True:
            choice = input('Would you like to play another round (yes/no)?: ')
            if choice == 'yes':
                game_on = True
                break
            elif choice == 'no':
                game_on = False
                break
            else:
                print('Invalid input. Please enter yes or no.')
    else:
        print('You are out of funds. Game over.')
        break

What is your name: Mike
Your bankroll balance is 100.
Your current balance is: 100. How much do you want to bet?: 100

You hold 
9 of Diamonds
Ace of Diamonds
The total hand value is 20.
Do you want to hit or stay? hit
You were dealt Queen of Spades.
Adjusted for an ace!

You hold 
9 of Diamonds
Ace of Diamonds
Queen of Spades
The total hand value is 20.
Do you want to hit or stay? hit
You were dealt 6 of Spades.
Bust!
True

Now it's the dealer's turn.

The dealer holds 
4 of Hearts
4 of Clubs
The total hand value is 8.
The dealer was dealt 6 of Clubs.

The dealer holds 
4 of Hearts
4 of Clubs
6 of Clubs
The total hand value is 14.
The dealer was dealt 3 of Clubs.

The dealer holds 
4 of Hearts
4 of Clubs
6 of Clubs
3 of Clubs
The total hand value is 17.
The dealer was dealt 3 of Hearts.

The dealer holds 
4 of Hearts
4 of Clubs
6 of Clubs
3 of Clubs
3 of Hearts
The total hand value is 20.
The dealer was dealt 8 of Clubs.

The dealer holds 
4 of Hearts
4 of Clubs
6 of Clubs
3 of Clubs


In [13]:
if skip:
    print(skip)

In [11]:
skip

False

In [14]:
print(player1, player1_hand, '\n', dealer, dealer_hand)

Player: Mike, Bankroll Balance: 200. 
9 of Diamonds
Ace of Diamonds
Queen of Spades
6 of Spades
The total hand value is 26 
 Dealer 
4 of Hearts
4 of Clubs
6 of Clubs
3 of Clubs
3 of Hearts
8 of Clubs
The total hand value is 28
